## Expression Trees and Evaluation

We are going to explore the idea of expression trees and how they relate to
our tree structures, namely `TreeNode`, and to evaluate the expression trees
by rewriting the nodes in post-order traversal.

First, let's define our expression tree.

In [5]:
from AlgoTree.treenode import TreeNode
import json

# Define the expression tree
root = TreeNode(name="+", value="+", type="op")
root_1 = TreeNode(name="max", value="max", type="op", parent=root)
root_2 = TreeNode(name="+", value="+", type="op", parent=root)
root_1_1 = TreeNode(name="+", value="+", type="op", parent=root_1)
root_1_1_1 = TreeNode(name="var", value="x", type="var", parent=root_1_1)
root_1_1_2 = TreeNode(name="const", value=1, type="const", parent=root_1_1)
root_2_1 = TreeNode(name="max", value="max", type="op", parent=root_2)
root_2_1_1 = TreeNode(name="var", value="x", type="var", parent=root_2_1)
root_2_1_2 = TreeNode(name="var", value="y", type="var", parent=root_2_1)
root_2_2 = TreeNode(name="const", value=3, type="const", parent=root_2)
root_2_3 = TreeNode(name="var", value="y", type="var", parent=root_2)

json_tree = """
    {
    "value": "+",
    "type": "op",
    "children": [
        {
            "value": "max",
            "type": "op",
            "children": [
                {
                    "value": "+",
                    "type": "op",
                    "children": [
                        {"type": "var", "value": "x"},
                        {"type": "const", "value": 1},
                    ],
                },
                {"type": "const", "value": 0},
            ],
        },
        {
            "type": "op",
            "value": "+",
            "children": [
                {
                    "type": "op",
                    "value": "max",
                    "children": [
                        {"type": "var", "value": "x"},
                        {"type": "var", "value": "y"},
                    ],
                },
                {"type": "const", "value": 3},
                {"type": "var", "value": "y"},
            ],
        },
    ],
}
"""

In [6]:
# Print the expression tree in JSON format
print(json.dumps(root, indent=2))

{
  "value": "+",
  "type": "op",
  "__name__": "+",
  "children": [
    {
      "value": "max",
      "type": "op",
      "__name__": "max",
      "children": [
        {
          "value": "+",
          "type": "op",
          "__name__": "+",
          "children": [
            {
              "value": "x",
              "type": "var",
              "__name__": "var"
            },
            {
              "value": 1,
              "type": "const",
              "__name__": "const"
            }
          ]
        }
      ]
    },
    {
      "value": "+",
      "type": "op",
      "__name__": "+",
      "children": [
        {
          "value": "max",
          "type": "op",
          "__name__": "max",
          "children": [
            {
              "value": "x",
              "type": "var",
              "__name__": "var"
            },
            {
              "value": "y",
              "type": "var",
              "__name__": "var"
            }
          ]
      

### Visualizing the Tree Structure

We can use the `TreeViz` class to visualize the tree structure.

In [3]:
from AlgoTree.tree_converter import TreeConverter
from AlgoTree.tree_viz import TreeViz

In [7]:
# Visualize the tree using TreeViz
print(TreeViz.text(root.node("max"), node_name=lambda x: x.value))

AttributeError: node must have a 'children' property

In [8]:
print(TreeViz.text2(root.node("var"), node_name=lambda x: x.value))
print(TreeViz.text2(root.node("max").node("var"), node_name=lambda x: x.value))
#print(root.subtree("var"))

AttributeError: node must have a 'children' property

Here is what that looks like in a more convenient form.

In [ ]:
# Generate and save a visual representation of the tree
TreeViz.image(root, node_name=lambda n: n.type + ": " + str(n.value),
              filename="./images/eval/tree-expr.png")

Here is an image of the local `tree-expr.png` file just generated:

![](./images/eval/tree-expr.png)


### Post-order Traversal

As a tree structure, `TreeNode` implements an interface that permits
tree traversal algorithms like depth-first pre-order and post-order traversals.

We are going to implement a simple post-order traversal algorithm to permit
computation of the expression tree we defined earlier, `expr`. We see that
it contains three operator types, `+`, `*`, and `max`, as well as numbers and variables.

We will provide a **closure** over all of these types so that when we evaluate
the expression in post-order, all of the types are defined for the operations.

In [ ]:
def postorder(node, fn, ctx):
    """
    Applies function `fn` to the nodes in the tree using post-order traversal.
    :param fn: Function to apply to each node. Should accept one argument: the node.
    :param ctx: Context passed to the function.
    :return: The tree with the function `fn` applied to its nodes.
    """
    results = []
    for child in node.children:
        result = postorder(child, fn, ctx)
        if result is not None:
            results.append(result)

    node.children = results
    return fn(node, ctx)

The function `postorder` takes a tree node `node`, a function `fn`, and a context `ctx`, and returns a rewritten tree.

At each node, `postorder` recursively calls `fn` on its children before applying `fn` to the node itself. This is the essence of post-order traversal.

Post-order is useful for problems where the children need to be processed before the node itself. For example, evaluating an expression tree, where typically the value of a node can only be computed after the values of its children are known.

In contrast, pre-order traversal applies `fn` to the node before applying it to the children. Pre-order may be useful for tasks such as rewriting the tree in a different form, like algebraic simplification.

### Expression Tree Evaluator

We will now design a simple expression tree evaluator, `Eval`.

In [ ]:
from copy import deepcopy
import uuid
from AlgoTree.flattree_node import FlatTreeNode

class Eval:
    """
    An evaluator for expressions defined by operations on types, respectively
    defined by `Eval.Op` and `Eval.Type`. The operations are a
    dictionary where the keys are the operation names and the values are
    functions that take a node and a context and return the value of the
    operation in that context.
    """

    Op = {
        "+": lambda x: sum(x),
        "max": lambda x: max(x),
    }

    Type = {
        "const": lambda node, _: node["value"],
        "var": lambda node, ctx: ctx[node["value"]],
        "op": lambda node, _: Eval.Op[node["value"]](
            [c["value"] for c in node.children]
        ),
    }

    def __init__(self, debug=True):
        """
        :param debug: If True, print debug information
        """
        self.debug = debug

    def __call__(self, expr, ctx):
        NodeType = type(expr)
        def _eval(node, ctx):
            expr_type = node["type"]
            value = Eval.Type[expr_type](node, ctx)
            result = NodeType(type="const", value=value)
            if self.debug:
                print(f"Eval({node.payload}) -> {result.payload}")
            return result

        return postorder(deepcopy(expr), _eval, ctx)

To evaluate an expression tree, we need the operations to be defined for all
of the types during post-order (bottom-up) traversal. We can define a
closure over all of the types, and then use that closure to evaluate the
expression tree.

We call this closure a context. Normally, the operations and other things
are also defined in the closure, but for simplicity we will just define the
operations and provide closures over the variables.

In [ ]:
# Define the context with variable values
ctx = {"x": 1, "y": 2, "z": 3}

# Evaluate the expression tree with the context
result = Eval(debug=True)(root, ctx)

Eval({'value': 'x', 'type': 'var'}) -> {'type': 'const', 'value': 1}
Eval({'value': 1, 'type': 'const'}) -> {'type': 'const', 'value': 1}
Eval({'value': '+', 'type': 'op'}) -> {'type': 'const', 'value': 2}
Eval({'value': 'max', 'type': 'op'}) -> {'type': 'const', 'value': 2}
Eval({'value': 'x', 'type': 'var'}) -> {'type': 'const', 'value': 1}
Eval({'value': 'y', 'type': 'var'}) -> {'type': 'const', 'value': 2}
Eval({'value': 'max', 'type': 'op'}) -> {'type': 'const', 'value': 2}
Eval({'value': 3, 'type': 'const'}) -> {'type': 'const', 'value': 3}
Eval({'value': 'y', 'type': 'var'}) -> {'type': 'const', 'value': 2}
Eval({'value': '+', 'type': 'op'}) -> {'type': 'const', 'value': 7}
Eval({'value': '+', 'type': 'op'}) -> {'type': 'const', 'value': 9}


In [ ]:
# Print the result of the evaluation
print(result)

TreeNode(, payload={'type': 'const', 'value': 9})


We see that we get the expected result, `9`. Note that it is still a tree, but
it has been transformed into a so-called self-evaluating tree expression,
which in this case is a single node with no children.

We can evaluate it again, and we see that it cannot be rewritten further. We
call this state a **normal form**. Essentially, we can think of the
tree as a program that computes a value, and the normal form is the result of
running the program.

In [ ]:
# Ensure the evaluated result is in its normal form
assert Eval(debug=False)(result, ctx).value == result.value

### Converting to FlatTree

Let's convert the tree to a `FlatTree` and perform the same evaluation.

In [ ]:
# Convert TreeNode to FlatTreeNode
flat_expr = TreeConverter.convert(source_node=root,
                                  target_type=FlatTreeNode,
                                  node_name=lambda _: str(uuid.uuid4()),
                                  extract=lambda n: n.payload)
print(json.dumps(flat_expr.tree, indent=4))

{
    "6edfd289-267c-4195-9763-3ba5c1a63518": {
        "value": "+",
        "type": "op"
    },
    "e860c7f8-ec77-4a98-ba95-03240da1351e": {
        "value": "max",
        "type": "op",
        "parent": "6edfd289-267c-4195-9763-3ba5c1a63518"
    },
    "9ac4a6cf-673a-4c1c-b3b7-901ac101da20": {
        "value": "+",
        "type": "op",
        "parent": "e860c7f8-ec77-4a98-ba95-03240da1351e"
    },
    "32d94df6-3400-4281-9c27-8868fea7048c": {
        "value": "x",
        "type": "var",
        "parent": "9ac4a6cf-673a-4c1c-b3b7-901ac101da20"
    },
    "622c6ef2-dba8-4c1d-9df4-712b2c8c2067": {
        "value": 1,
        "type": "const",
        "parent": "9ac4a6cf-673a-4c1c-b3b7-901ac101da20"
    },
    "941de746-df29-4243-99b2-ac3ae311422a": {
        "value": "+",
        "type": "op",
        "parent": "6edfd289-267c-4195-9763-3ba5c1a63518"
    },
    "8023a736-d9d8-4f60-ac0d-07d2fecfbcf1": {
        "value": "max",
        "type": "op",
        "parent": "941de746-df29-424

In [ ]:
# Evaluate the flat tree expression
result = Eval(debug=True)(flat_expr, ctx)
# Print the result of the evaluation
print(result)
# Print the underlying flat tree structure
print(json.dumps(result.tree, indent=4))

Eval({'value': 'x', 'type': 'var'}) -> {'type': 'const', 'value': 1}
Eval({'value': 1, 'type': 'const'}) -> {'type': 'const', 'value': 1}
Eval({'value': '+', 'type': 'op'}) -> {'type': 'const', 'value': 2}
Eval({'value': 'max', 'type': 'op'}) -> {'type': 'const', 'value': 2}
Eval({'value': 'x', 'type': 'var'}) -> {'type': 'const', 'value': 1}
Eval({'value': 'y', 'type': 'var'}) -> {'type': 'const', 'value': 2}
Eval({'value': 'max', 'type': 'op'}) -> {'type': 'const', 'value': 2}
Eval({'value': 3, 'type': 'const'}) -> {'type': 'const', 'value': 3}
Eval({'value': 'y', 'type': 'var'}) -> {'type': 'const', 'value': 2}
Eval({'value': '+', 'type': 'op'}) -> {'type': 'const', 'value': 7}
Eval({'value': '+', 'type': 'op'}) -> {'type': 'const', 'value': 9}
FlatTreeNode(name=f6a37f95-ba1f-4e00-a6cf-32d83785deb9, parent=None, payload={'type': 'const', 'value': 9})
{
    "f6a37f95-ba1f-4e00-a6cf-32d83785deb9": {
        "type": "const",
        "value": 9
    }
}


The `FlatTree` structure is a different kind of tree structure that is more
convenient for relatively flatter data, like conversation logs. It is a tree
structure that is flattened into a dictionary of key-value pairs, where the
value is also a dictionary. This value dictionary optionally contains the parent
key, and if not then it is a child of a so-called logical root.